In [1]:
%pip install transformers torchaudio librosa soundfile


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 324.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 259.3 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 734.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 223.3 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 196.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 232.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.2/26.2 MB 165.1 kB/s eta 0:00:0000:0100:05

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Step 1: Import required libraries
from transformers import AutoProcessor, AutoModelForAudioClassification
import torch
import librosa
import os

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Step 2: Load the pre-trained model and processor from Hugging Face
model_name = "MelodyMachine/Deepfake-audio-detection-V2"

# Load processor: responsible for preparing the raw audio for the model
processor = AutoProcessor.from_pretrained(model_name)

# Load model: pre-trained for detecting deepfake (FAKE vs REAL) in audio
model = AutoModelForAudioClassification.from_pretrained(model_name)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:61: FutureWarning: Loading a tokenizer inside Wav2Vec2Processor from a config that does not include a `tokenizer_class` attribute is deprecated and will be removed in v5. Please add `'tokenizer_class': 'Wav2Vec2CTCTokenizer'` attribute to either your `config.json` or `tokenizer_config.json` file to suppress this warning: 
  warnings.warn(


OSError: Can't load tokenizer for 'MelodyMachine/Deepfake-audio-detection-V2'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'MelodyMachine/Deepfake-audio-detection-V2' is the correct path to a directory containing all relevant files for a Wav2Vec2CTCTokenizer tokenizer.

In [ ]:
# Step 3: Function to predict if audio is FAKE or REAL
def detect_audio_fake(audio_path):
    # Load audio file using librosa (resample to 16kHz as required by model)
    waveform, sample_rate = librosa.load(audio_path, sr=16000)

    # Process audio for model input
    inputs = processor(waveform, sampling_rate=16000, return_tensors="pt")

    # Inference using model (no gradient needed for evaluation)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Get prediction
    predicted_class_id = torch.argmax(logits, dim=-1).item()
    predicted_label = model.config.id2label[predicted_class_id]

    # Get confidence score (softmax)
    probabilities = torch.nn.functional.softmax(logits, dim=-1)[0]
    confidence = probabilities[predicted_class_id].item()

    return predicted_label, confidence

In [ ]:
# Step 4: Run prediction on an audio file
if __name__ == "__main__":
    audio_file = "sample_audio.wav"  # Change this to your audio file path

    if os.path.exists(audio_file):
        label, confidence = detect_audio_fake(audio_file)
        print(f"\n🔍 Prediction: {label}")
        print(f"✅ Confidence: {confidence:.2f}")
    else:
        print(f"❌ Audio file not found: {audio_file}")

In [7]:
from transformers import pipeline

# Load the pre-trained pipeline
audio_classifier = pipeline("audio-classification", model="MelodyMachine/Deepfake-audio-detection-V2")

# Provide the path to your audio file (16kHz WAV format recommended)
audio_path = "/Users/fenilvadher/Documents/Collage Data/SEM - 6/AI/AI Project/fake_audio/real/speaker1_1.wav"

# Run the pipeline
results = audio_classifier(audio_path)

# Print results
print("Audio Classification Results:")
for res in results:
    print(f"Label: {res['label']}, Score: {res['score']:.4f}")


Device set to use mps:0


Audio Classification Results:
Label: real, Score: 1.0000
Label: fake, Score: 0.0000
